In [ ]:
!pip install datasets evaluate accelerate
!pip install git+https://github.com/huggingface/transformers

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import shutil
shutil.copy("/content/drive/MyDrive/Masterarbeit/fantastic-umbrella/run_glue_no_trainer_modded.py", "/content/run_glue_no_trainer_modded.py")

In [ ]:
import os
import torch
print(f'torch Version {torch.__version__}')
import transformers
print(f'transformers Version {transformers.__version__}')

Declare all possible parameters

In [25]:
possible_values = {
    "task" : ['wnli', 'rte'],
    "learning_rate" : [5e-5, 1e-5],
    "hidden_dropout" : [0.1, 0.5, 0.8],
    "use_pretrain_dropout_in_first_pass" : [True, False],
    "train_data_frac" : [1, 0.5, 0.1],
    "use_modded" : [True, False],
    "num_train_epochs" : [5, 10, 20]
}

Create an exhaustive list of all parameter combinations

In [ ]:
from itertools import product
keys, values = zip(*possible_values.items())
permutations_dicts = [dict(zip(keys, v)) for v in product(*values)]
print(len(permutations_dicts))
permutations_dicts

In [1]:
# Load the TensorBoard notebook extension
%load_ext tensorboard
%tensorboard --logdir "/content/drive/MyDrive/Masterarbeit/fantastic-umbrella/runs/"

ModuleNotFoundError: No module named 'tensorboard'

Construct the script command

In [32]:
import datetime
import os
for iteration, parameters in enumerate(permutations_dicts):
    print(f'##################################### Current iteration: {iteration} #####################################')
    base_command = 'python run_glue_no_trainer_modded.py '
    argument_list = ["--model_name_or_path", "bert-base-cased",
                     "--max_length", "128",
                     "--per_device_train_batch_size", "32",
                     "--seed", "0",
                     "--with_tracking",
                     "--report_to tensorboard"
                    ]



    argument_list = argument_list + ["--task_name", f'{parameters["task"]}']
    argument_list = argument_list + ["--num_train_epochs", f'{parameters["num_train_epochs"]}']
    argument_list = argument_list + ["--learning_rate", f'{parameters["learning_rate"]}']
    argument_list = argument_list + ["--train_data_frac", f'{parameters["train_data_frac"]}']
    argument_list = argument_list + ["--hidden_dropout", f'{parameters["hidden_dropout"]}']


    if parameters["use_modded"]:
        argument_list.append("--use_modded")

    if parameters["use_pretrain_dropout_in_first_pass"]:
        argument_list.append("--use_pretrain_dropout_in_first_pass")
    
    folder_name = 'run-{date:%Y%m%d_%H%M%S}'.format( date=datetime.datetime.now() )
    output_dir = f"/content/drive/MyDrive/Masterarbeit/fantastic-umbrella/runs/{folder_name}"
    argument_list = argument_list + ["--output_dir",output_dir]
    # print(argument_list)
    argument_string = ' '.join(argument_list)
    # print(argument_string)
    instruction = base_command + argument_string
    # print(instruction)

    # run the script with the desired arguments
    # os.system(instruction) # vscode style
    !{instruction} # colab style

['--model_name_or_path', 'bert-base-cased', '--max_length', '128', '--per_device_train_batch_size', '32', '--seed', '0', '--with_tracking', '--report_to tensorboard', '--task_name', 'WNLI', '--num_train_epochs', '5', '--learning_rate', '5e-05', '--train_data_frac', '1', '--hidden_dropout', '0.1', '--use_modded', '--use_pretrain_dropout_in_first_pass', '--output_dir', '/content/drive/MyDrive/Masterarbeit/fantastic-umbrella/runs/run-20230614_121442']


KeyboardInterrupt: 

In [6]:
!bash run_modded.sh

Der Befehl "bash" ist entweder falsch geschrieben oder
konnte nicht gefunden werden.


In [ ]:
import os
import fnmatch

for path,_,files in os.walk('/content'):
    for file in fnmatch.filter(files,'pytorch_model.bin'):
        fullname = os.path.abspath(os.path.join(path,file))
        os.remove(fullname)
        print(f'removed {fullname}')

In [ ]:
# Zip the modded runs
!zip -r '/content/mod_runs.zip' '/content/mod_runs'


In [ ]:
!bash run_vanilla.sh

In [ ]:
# Zip the vanilla runs
!zip -r '/content/vanilla_runs.zip' '/content/vanilla_runs'

In [ ]:
from google.colab import files
files.download('/content/vanilla_runs.zip') 